In [1]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import glm
import collections
from statsmodels.sandbox.stats.multicomp import multipletests


In [2]:
df=pd.read_csv('Final_poisson.csv').iloc[:,1:]

In [3]:
import time

In [4]:
def conteo_perf(newdf=df):
    df_drop=newdf.drop_duplicates(subset='LAT_LON') 
    conteos=newdf[['LAT_LON','SITIO_EXACTO_ACCIDENTE']].groupby(['LAT_LON','SITIO_EXACTO_ACCIDENTE']).size().reset_index(name='count')
    grp_conteos = conteos.groupby('LAT_LON')['count'].agg(['sum','max'])
    grp_conteos['sum_rate'] = grp_conteos['max'] / grp_conteos['sum']
    filtered_conteos = grp_conteos[grp_conteos['sum_rate'] > 0.3]
    return_df = filtered_conteos.drop(['max', 'sum_rate'], axis=1)
    return_df.rename(columns = {'sum':'NACCIDENTS'}, inplace = True) 
    fusion=pd.merge(return_df,df_drop,on='LAT_LON',how='inner')[['NACCIDENTS','AÑO_ACCIDENTE','MES_ACCIDENTE',
                                                     'CLASE_ACCIDENTE','LONGITUD', 'LATITUD',
                                                     'POT', 'PIEZA_URBANA', 'BUS_SUBIDA','BUS_BAJADA', 
                                                     'TRAFICO','ES_FESTIVO','SOLO_HORA']].reset_index().iloc[:,1:]
    return(fusion)

start_time = time.time()
b=conteo_perf()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.12785577774047852 seconds ---


In [5]:
semaforos=pd.read_csv('lights_clean.csv')[['LATITUD','LONGITUD']]

In [6]:
print(semaforos.columns)
print(b.columns)

Index(['LATITUD', 'LONGITUD'], dtype='object')
Index(['NACCIDENTS', 'AÑO_ACCIDENTE', 'MES_ACCIDENTE', 'CLASE_ACCIDENTE',
       'LONGITUD', 'LATITUD', 'POT', 'PIEZA_URBANA', 'BUS_SUBIDA',
       'BUS_BAJADA', 'TRAFICO', 'ES_FESTIVO', 'SOLO_HORA'],
      dtype='object')


In [17]:
def distancia_auxiliar(x,lights=semaforos):
    dx = (lights['LONGITUD']-x[1])*40000*np.cos((lights['LATITUD']+x[0])*math.pi/360)/360
    dy = (lights['LATITUD']-x[0])*40000/360
    dist=np.sqrt(dx**2+dy**2)
    
    return([np.min(dist),np.sum(dist<0.1), np.sum((dist>=0.1) & (dist<0.5)), np.sum((dist>=0.5) & (dist<2))])
    
def distancia_semaforos_mas_lento(df=b[['LATITUD','LONGITUD']],lights=semaforos):
    aux=df[['LATITUD','LONGITUD']].values
    return([distancia_auxiliar(x) for x in aux])
    #return(df.apply(distancia_auxiliar,axis=1))
#     return([np.min(distancia_auxiliar(x=[aux1,aux2])) for (aux1,aux2) in zip(df[['LATITUD'],df['LONGITUD']])])
#     for i in range(df.shape[0]):
#         aux=df.iloc[i,:][['LATITUD','LONGITUD']]
#         dx = (lights['LONGITUD']-aux['LONGITUD'])*40000*np.cos((lights['LATITUD']+aux['LATITUD'])*math.pi/360)/360
#         dy = (lights['LONGITUD']-aux['LATITUD'])*40000/360
#         dist=np.sqrt(dx**2+dy**2)
#         np.min(dist)

In [18]:
start_time = time.time()
a=distancia_semaforos_mas_lento()
print("--- %s seconds ---" % (time.time() - start_time))

--- 22.063221216201782 seconds ---


In [15]:
def distancia_semaforos_mas_rapido(final_df=b,lights=semaforos):
    closest=np.zeros(final_df.shape[0])
    numclose=np.zeros(final_df.shape[0])
    numclose2=np.zeros(final_df.shape[0])
    numclose3=np.zeros(final_df.shape[0])
    for i in range(final_df.shape[0]):
        corner_lat=final_df['LATITUD'][i]
        corner_lon=final_df['LONGITUD'][i]
        light_lat=np.array(lights['LATITUD'])
        light_lon=np.array(lights['LONGITUD'])
        dx = (light_lon-corner_lon)*40000*np.cos((light_lat+corner_lat)*math.pi/360)/360
        dy = (light_lat-corner_lat)*40000/360
        distance=np.sqrt(dx**2+dy**2)
        closest[i]=distance[distance.argmin()]
        numclose[i]=sum(distance<0.1)
        numclose2[i]=sum(((distance>=0.1) & (distance<0.5)))
        numclose3[i]=sum(((distance>=0.5) & (distance<2)))
    final_df['NLIGHTS']=list(numclose)
    final_df['NLIGHTS2']=list(numclose2)
    final_df['NLIGHTS3']=list(numclose3)

In [16]:
start_time = time.time()
distancia_semaforos_mas_rapido()
print("--- %s seconds ---" % (time.time() - start_time))

--- 13.806360960006714 seconds ---


In [75]:
b

,NACCIDENTS,AÑO_ACCIDENTE,MES_ACCIDENTE,CLASE_ACCIDENTE,LONGITUD,LATITUD,POT,PIEZA_URBANA,BUS_SUBIDA,BUS_BAJADA,TRAFICO,ES_FESTIVO,SOLO_HORA,NLIGHTS,NLIGHTS2,NLIGHTS3
0,1,2019,4,Choque,-74.810748,10.915367,protegida,ninguna,0,0,0.003922,YES,11,0.0,0.0,0.0
1,1,2018,5,Choque,-74.795213,10.917122,protegida,ninguna,3,0,0.003922,NO,14,0.0,0.0,3.0
2,1,2018,1,Choque,-74.807522,10.917998,protegida,ninguna,2,0,0.705882,NO,11,0.0,0.0,3.0
3,1,2019,6,Choque,-74.797185,10.918646,protegida,ninguna,2,0,0.003922,NO,15,0.0,0.0,3.0
4,1,2019,5,Choque,-74.826326,10.921112,protegida,ninguna,0,0,0.294118,NO,13,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5825,1,2016,9,Choque,-74.827026,11.040540,residencial,ribera occidental,1,0,0.705882,NO,11,0.0,0.0,0.0
5826,1,2018,4,Atropello,-74.824731,11.040967,residencial,ribera occidental,1,0,0.003922,NO,20,0.0,0.0,0.0
5827,1,2017,5,Atropello,-74.827933,11.041935,residencial,ribera occidental,1,0,0.705882,NO,15,0.0,0.0,0.0
5828,1,2018,1,Choque,-74.826731,11.042032,residencial,ribera occidental,1,0,0.701961,NO,9,0.0,0.0,0.0
